In [1]:
import tensorflow as tf
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased', do_lower_case=False)
import batch_processing
import re
import pandas as pd
import numpy as np
import math

import batch_processing
import fonctions

import gzip
import pickle as pkl
import re
import collections
import itertools
import time

import multiprocessing as mp
from multiprocessing.pool import ThreadPool

from os import listdir
from tqdm.notebook import tqdm
from os.path import isfile, join

/home/maxime/anaconda3/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/maxime/anaconda3/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/maxime/anaconda3/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/maxime/anaconda3/lib/python3.6/site-packages/tensorflow/python/framework/dty

In [2]:
def trend_processing(x):
    if type(x) is str:
        return re.split(r'\t+', x)
    return float('Nan')

def get_trends(directory, doc_name, init):
    
    all_features = ["text_tokens", "hashtags", "tweet_id", 
                    "present_media", "present_links", 
                    "present_domains", "tweet_type","language", 
                    "tweet_timestamp", "engaged_with_user_id",
                    "engaged_with_user_follower_count", "engaged_with_user_following_count", 
                    "engaged_with_user_is_verified", "engaged_with_user_account_creation",
                    "engaging_user_id", "engaging_user_follower_count", 
                    "engaging_user_following_count", "engaging_user_is_verified",
                    "engaging_user_account_creation", "engagee_follows_engager"]
    
    df = pd.read_csv(directory+doc_name, encoding="utf-8", sep='\x01', header=None)
    
    labels = ['reply_timestamp','retweet_timestamp', 'retweet_with_comment_timestamp','like_timestamp']
    all_variables = all_features + labels
    df.columns = all_variables
    df.index = df.tweet_id+'-'+df.engaging_user_id
        
    df['reply_timestamp']=[ 0 if math.isnan(x) else 1 for x in df['reply_timestamp'] ]
    df['retweet_timestamp']=[ 0 if math.isnan(x) else 1 for x in df['retweet_timestamp'] ]
    df['retweet_with_comment_timestamp']=[ 0 if math.isnan(x) else 1 for x in df['retweet_with_comment_timestamp'] ]
    df['like_timestamp']=[ 0 if math.isnan(x) else 1 for x in df['like_timestamp'] ]
            
    df = df.drop(['tweet_id', 'hashtags','present_media','present_links','engaging_user_id','engaged_with_user_id',
                  'present_domains','tweet_timestamp',
                 'engaged_with_user_account_creation', 'engaging_user_account_creation'],axis=1)
    
    df['text_tokens'] = [ trend_processing(x) for x in df['text_tokens'] ] 
    labels= ['like_timestamp','retweet_timestamp','retweet_with_comment_timestamp','reply_timestamp']      
    df = df.filter(labels+['text_tokens'], axis=1)
    
    return df

def text_influence_on_chunk(chunk, chunk_id):
    
    bert_ratio = {}
    buff_bert_ratio = {}
    engagements = ['like_timestamp','retweet_timestamp','retweet_with_comment_timestamp','reply_timestamp']
    directory = '/home/maxime/Desktop/RecSys2020/data/batches/'

    iteration=1
    for batch_file in chunk:
        
        df = get_trends(directory,batch_file,True)
        df = df [ df['text_tokens'].isna()==False ]
        bert = np.unique ( list( itertools.chain.from_iterable(df.text_tokens) ) )
        
        select = { k:[0,0,0,0] for k in bert if k not in buff_bert_ratio.keys()  }
        buff_bert_ratio.update(select)
        
        for idx, engagement in enumerate(engagements):
            buff =  df[ df[engagement]==1 ]
            buff_presence = collections.Counter( list( itertools.chain.from_iterable(buff.text_tokens) ) )
            { update_eng(buff_bert_ratio, k, v, idx) for k,v in buff_presence.items() }
        
        if iteration in [4, 50, 100, 150, 200, 250, 300, 350, 400, 450, 500, 550]:
            
            select = {k:v for k,v in buff_bert_ratio.items() if k in bert_ratio.keys() }
            reste = {k:v for k,v in buff_bert_ratio.items() if k not in bert_ratio.keys() }
            { update_agg(bert_ratio, k, v) for k,v in select.items() }
            bert_ratio.update(reste)
            buff_bert_ratio={}
            print(len( bert_ratio.keys() ))

        print(iteration)
        iteration=iteration+1
        
    select = {k:v for k,v in buff_bert_ratio.items() if k in bert_ratio.keys() }
    reste = {k:v for k,v in buff_bert_ratio.items() if k not in bert_ratio.keys() }
    { update_agg(bert_ratio, k, v) for k,v in select.items() }
    bert_ratio.update(reste)
    
    print('cutting and saving...')
    
    with gzip.open('/home/maxime/Desktop/RecSys2020/trends/bert_ratio_{}.pkl.gz'.format(chunk_id), 'wb') as f:
        pkl.dump(bert_ratio,f)
            
    return True

def update_eng(bert_ratio, key,v, idx):
    bert_ratio[key][idx]= bert_ratio[key][idx]+v

def update_agg(bert_ratio, k, v):
    bert_ratio[k]=[x + y for x, y in zip(bert_ratio[k], v )]
    

In [3]:
%%time

batch_path='/home/maxime/Desktop/RecSys2020/data/batches'
batch_list = [f for f in listdir(batch_path) if isfile(join(batch_path, f))]
chunks = fonctions.chunkIt(batch_list, 8)

if __name__ == '__main__':
    
    # Setup a list of processes that we want to run
    processes = [ mp.Process(target=text_influence_on_chunk, args=(chunk, chunk_id) ) for chunk_id, chunk in enumerate(chunks) ]

    # Run processes
    for p in processes:
        p.start()
        
    #Stop the processes
    for p in processes:
        p.join() 

1
1
1
1
1
1
1
1
2
2
2
2
2
2
2
2
3
3
3
3
3
3
3
3
64306
4
64594
4
64339
64008
4
64785
4
4
64016
4
64075
4
64251
4
5
5
5
5
5
5
5
5
6
6
6
6
6
6
6
6
7
7
7
7
7
7
7
7
8
8
8
8
8
8
8
8
9
9
9
9
9
9
9
9
10
10
10
10
10
10
10
10
11
11
11
11
11
11
11
11
12
12
12
12
12
12
12
12
13
13
13
13
13
13
13
13
14
14
14
14
14
14
14
14
15
15
15
15
15
15
15
15
16
16
16
16
16
16
16
16
17
17
17
17
17
17
17
17
18
18
18
18
18
18
18
18
19
19
19
19
19
19
19
19
20
20
20
20
20
20
20
20
21
21
21
21
21
21
21
21
22
22
22
22
22
22
22
22
23
23
23
23
23
23
23
23
24
24
24
24
24
24
24
24
25
25
25
25
25
25
25
25
26
26
26
26
26
26
26
26
27
27
27
27
27
27
27
27
28
28
28
28
28
28
28
28
29
29
29
29
29
29
29
29
30
30
30
30
30
30
30
30
31
31
31
31
31
31
31
31
32
32
32
32
32
32
32
32
33
33
33
33
33
33
33
33
34
34
34
34
34
34
34
34
35
35
35
35
35
35
35
35
36
36
36
36
36
36
36
36
37
37
37
37
37
37
37
37
38
38
38
38
38
38
38
38
39
39
39
39
39
39
39
39
40
40
40
40
40
40
40
40
41
41
41
41
41
41
41
41
42
42
42
42
42
42
42
42
43
43
43
43
43
4

273
274
274
276
274
275
276
277
274
275
275
277
275
276
277
278
275
276
276
278
276
277
278
279
276
277
277
279
277
279
278
280
277
278
280
278
278
279
280
281
278
279
279
281
280
279
281
282
279
280
282
280
281
280
282
283
280
283
281
281
281
283
282
284
281
284
282
282
282
283
285
284
282
283
285
283
283
284
285
286
283
284
286
284
285
284
286
287
284
285
285
287
287
285
286
288
285
286
288
286
287
286
288
289
286
287
289
287
288
289
287
290
287
288
288
290
289
290
288
291
288
289
289
291
291
290
289
292
289
290
290
292
290
292
293
291
290
291
293
291
291
293
292
294
291
294
292
292
292
294
293
295
292
295
293
293
293
295
294
296
293
294
296
294
294
296
295
297
294
295
297
295
295
297
298
296
295
296
298
296
296
298
297
299
296
297
299
297
297
299
298
100822
300
297
298
298
100917
300
298
299
298
100745
300
301
299
299
299
301
302
301
100870
300
299
100846
300
302
100829
300
100809
300
303
302
101023
300
301
301
303
301
304
301
303
302
301
302
304
302
305
302
304
303
302
303
305
303


In [4]:
engagements = ['like_timestamp','retweet_timestamp','retweet_with_comment_timestamp','reply_timestamp']
global_bert_ratio = {}

for chunk_id in range(8):
    
    print(chunk_id)
    
    with gzip.open('/home/maxime/Desktop/RecSys2020/trends/bert_ratio_{}.pkl.gz'.format(chunk_id), 'rb') as f:
        bert_ratio = pkl.load(f)
        
    select = {k:v for k,v in bert_ratio.items() if k in global_bert_ratio.keys() }
    reste = {k:v for k,v in bert_ratio.items() if k not in global_bert_ratio.keys() }
    { update_agg(global_bert_ratio, k, v) for k,v in select.items() }     
    global_bert_ratio.update(reste)

print( len(global_bert_ratio.keys() ) )

0
1
2
3
4
5
6
7
107171


In [5]:
def ratio_extraction(v):

    total_engagement = sum(v)

    if sum(v)==0:
        like_ratio = 0
        retweet_ratio = 0
        rtc_ratio = 0
        reply_ratio = 0
    else:
        like_ratio = round( v[0]/sum(v), 3)
        retweet_ratio = round(v[1]/sum(v),3)
        rtc_ratio = round(v[2]/sum(v) , 3)
        reply_ratio = round(v[3]/sum(v),3)

    return [total_engagement, like_ratio, retweet_ratio, rtc_ratio, reply_ratio ]

global_ratio2 = { k:ratio_extraction(v) for k,v in tqdm( global_bert_ratio.items() ) }
computed_ratio = {'like_timestamp':{},'retweet_timestamp':{},'retweet_with_comment_timestamp':{},'reply_timestamp':{}}

engagements = ['like_timestamp','retweet_timestamp','retweet_with_comment_timestamp','reply_timestamp']

for idx,engagement in enumerate(engagements):
    
    computed_ratio[engagement]= {k:(v[0],v[idx+1]) for k,v in tqdm(global_ratio2.items()) if v[0]>50000 and v[idx+1]>0.6 }


In [6]:
like_id = [ k for k in computed_ratio['like_timestamp'].keys() ]
print(len(like_id))
dictionnary_lk = { val:idx for idx,val in enumerate(like_id)}

retweet_id = [k for k in computed_ratio['retweet_timestamp'].keys() ]
print(len(retweet_id))
dictionnary_rt = { val:idx for idx,val in enumerate(retweet_id)}

rtc_id = [ k for k in computed_ratio['retweet_with_comment_timestamp'].keys() ]
print(len(rtc_id))
dictionnary_rtc = { val:idx for idx,val in enumerate(rtc_id)}

reply_id = [k for k in computed_ratio['reply_timestamp'].keys() ]
print(len(reply_id))
dictionnary_rpl = { val:idx for idx,val in enumerate(reply_id)}

with gzip.open('/home/maxime/Desktop/RecSys2020/trends/bert_influence.pkl.gz','wb') as f:
    pkl.dump(dictionnary_lk, f)
    pkl.dump(dictionnary_rt, f)


6474
190
0
0
